In [2]:
!python -V

Python 3.8.5


In [3]:
!jupyter nbconvert --to script DALLE-DeepSpeed.ipynb

[NbConvertApp] Converting notebook DALLE-DeepSpeed.ipynb to script
[NbConvertApp] Writing 18573 bytes to DALLE-DeepSpeed.py


In [4]:
!mv DALLE-DeepSpeed.py ../scripts/

In [63]:
from pathlib import Path
from random import randint, choice
import time
from glob import glob
import os
import shutil

import torch
import wandb  # Quit early if user doesn't have wandb installed.
from torch.nn.utils import clip_grad_norm_
from torch.utils.data import Dataset
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader

from dalle_pytorch import DiscreteVAE, DALLE
from dalle_pytorch import distributed_utils
# from dalle_pytorch.loader import TextImageDataset
from dalle_pytorch.tokenizer import tokenizer

# libraries needed for webdataset support
from torchvision import transforms as T
import PIL
from io import BytesIO

import datetime

In [7]:
torch.cuda.device_count()

1

In [53]:
def exists(val):
    return val is not None

def get_trainable_params(model):
    return [params for params in model.parameters() if params.requires_grad]

def cp_path_to_dir(cp_path, tag):
    """Convert a checkpoint path to a directory with `tag` inserted.
    If `cp_path` is already a directory, return it unchanged.
    """
    if not isinstance(cp_path, Path):
        cp_path = Path(cp_path)
    if cp_path.is_dir():
        return cp_path
    path_sans_extension = cp_path.parent / cp_path.stem
    cp_dir = Path(f'{path_sans_extension}-{tag}-cp')
    return cp_dir

In [65]:
# WEBDATASET_IMAGE_TEXT_COLUMNS = tuple(args.wds.split(','))
# ENABLE_WEBDATASET = True if len(WEBDATASET_IMAGE_TEXT_COLUMNS) == 2 else False
ENABLE_WEBDATASET = False

DALLE_OUTPUT_FILE_NAME = "../models/dalle-birds-deepspeed.pt"

VAE_PATH = "../models/vae-birds-final.pt"
# VQGAN_MODEL_PATH = args.vqgan_model_path
# VQGAN_CONFIG_PATH = args.vqgan_config_path
# DALLE_PATH = args.dalle_path
# RESUME = exists(DALLE_PATH)
RESUME = False

EPOCHS = 20
BATCH_SIZE = 4

LEARNING_RATE = 3e-4
GRAD_CLIP_NORM = 0.5
LR_DECAY = False
SAVE_EVERY_N_STEPS = 3000
KEEP_N_CHECKPOINTS = None

MODEL_DIM = 512
TEXT_SEQ_LEN = 256
DEPTH = 2
HEADS = 4
DIM_HEAD = 64
REVERSIBLE = True
LOSS_IMG_WEIGHT = 7
FF_DROPOUT = 0.0
ATTN_DROPOUT = 0.0
STABLE = False
SHIFT_TOKENS = True
ROTARY_EMB = True
FP16 = False

# ATTN_TYPES = tuple(args.attn_types.split(','))
ATTN_TYPES = None

DEEPSPEED_CP_AUX_FILENAME = 'aux_dalle-birds-deepspeed.pt'

IMAGE_FOLDER = '../data/CUB_200_2011/images'
TEXT_FOLDER = '../data/birds/text_c10'

RESIZE_RATIO = 0.75
TRUNCATE_CAPTIONS = False

In [10]:
assert Path(IMAGE_FOLDER).exists(), f'The path {IMAGE_FOLDER} was not found.'
assert Path(TEXT_FOLDER).exists(), f'The path {TEXT_FOLDER} was not found.'

In [11]:
class AttrDict(dict):
    def __init__(self, *args, **kwargs):
        super(AttrDict, self).__init__(*args, **kwargs)
        self.__dict__ = self
        
args = AttrDict()
args.deepspeed = True

In [46]:
required_env = ["RANK", "WORLD_SIZE", "MASTER_ADDR", "MASTER_PORT", "LOCAL_RANK"]
os.environ["RANK"] = "0"
os.environ["WORLD_SIZE"] = "1"
os.environ["MASTER_ADDR"] = "127.0.0.1"
os.environ["MASTER_PORT"] = "6006"
os.environ["LOCAL_RANK"] = "0"
list(map(lambda v: v in os.environ, required_env))

[True, True, True, True, True]

In [19]:
distr_backend = distributed_utils.set_backend_from_args(args)
distr_backend.initialize()

Using DeepSpeed for distributed execution


In [23]:
using_deepspeed = \
    distributed_utils.using_backend(distributed_utils.DeepSpeedBackend)
using_deepspeed

True

In [24]:
vae_path = Path(VAE_PATH)
assert vae_path.exists(), 'VAE model file does not exist'
assert not vae_path.is_dir(), \
    ('Cannot load VAE model from directory; please use a '
     'standard *.pt checkpoint. '
     'Currently, merging a DeepSpeed-partitioned VAE into a DALLE '
     'model is not supported.')

In [25]:
loaded_obj = torch.load(str(vae_path))

vae_params, weights = loaded_obj['hparams'], loaded_obj['weights']

vae = DiscreteVAE(**vae_params)
vae.load_state_dict(weights)

<All keys matched successfully>

In [26]:
IMAGE_SIZE = vae.image_size

dalle_params = dict(
    num_text_tokens=tokenizer.vocab_size,
    text_seq_len=TEXT_SEQ_LEN,
    dim=MODEL_DIM,
    depth=DEPTH,
    heads=HEADS,
    dim_head=DIM_HEAD,
    reversible=REVERSIBLE,
    loss_img_weight=LOSS_IMG_WEIGHT,
    attn_types=ATTN_TYPES,
    ff_dropout=FF_DROPOUT,
    attn_dropout=ATTN_DROPOUT,
    stable=STABLE,
    shift_tokens=SHIFT_TOKENS,
    rotary_emb=ROTARY_EMB,
)
resume_epoch = 0

In [27]:
def group_weight(model):
    group_decay, group_no_decay = [], []
    for params in model.named_parameters():
        if 'transformer' in params[0]:
            if 'bias' in params[0] or 'norm' in params[0]:
                group_no_decay.append(params[1])
                continue
        group_decay.append(params[1])

    assert len(list(model.parameters())) == len(group_decay) + len(group_no_decay)
    groups = [dict(params=group_decay), dict(params=group_no_decay, weight_decay=.0)]
    return groups

def imagetransform(b):
    return Image.open(BytesIO(b))

def tokenize(s):
    return tokenizer.tokenize(
        s.decode('utf-8'),
        TEXT_SEQ_LEN,
        truncate_text=args.truncate_captions).squeeze(0)

In [29]:
is_shuffle = not distributed_utils.using_backend(distributed_utils.HorovodBackend)
is_shuffle

True

In [30]:
imagepreproc = T.Compose([
            T.Lambda(lambda img: img.convert('RGB') if img.mode != 'RGB' else img),
            T.RandomResizedCrop(IMAGE_SIZE, scale = (RESIZE_RATIO, 1.), ratio = (1., 1.)),
            T.ToTensor()
        ])

In [31]:
class TextImageDataset(Dataset):
    def __init__(self,
                 image_folder,
                 text_folder,
                 text_len=256,
                 image_size=128,
                 truncate_captions=False,
                 resize_ratio=0.75,
                 tokenizer=None,
                 shuffle=False
                 ):
        """
        @param folder: Folder containing images and text files matched by their paths' respective "stem"
        @param truncate_captions: Rather than throw an exception, captions which are too long will be truncated.
        """
        super().__init__()
        self.shuffle = shuffle
        image_path = Path(image_folder)
        text_path = Path(text_folder)

        text_files = [*text_path.glob('**/*.txt')]
        image_files = [
            *image_path.glob('**/*.png'), *image_path.glob('**/*.jpg'),
            *image_path.glob('**/*.jpeg'), *image_path.glob('**/*.bmp')
        ]

        text_files = {text_file.stem: text_file for text_file in text_files}
        image_files = {image_file.stem: image_file for image_file in image_files}

        keys = (image_files.keys() & text_files.keys())

        self.keys = list(keys)
        self.text_files = {k: v for k, v in text_files.items() if k in keys}
        self.image_files = {k: v for k, v in image_files.items() if k in keys}
        self.text_len = text_len
        self.truncate_captions = truncate_captions
        self.resize_ratio = resize_ratio
        self.tokenizer = tokenizer
        self.image_transform = T.Compose([
            T.Lambda(lambda img: img.convert('RGB')
            if img.mode != 'RGB' else img),
            T.RandomResizedCrop(image_size,
                                scale=(self.resize_ratio, 1.),
                                ratio=(1., 1.)),
            T.ToTensor()
        ])
        
        

    def __len__(self):
        return len(self.keys)

    def random_sample(self):
        return self.__getitem__(randint(0, self.__len__() - 1))

    def sequential_sample(self, ind):
        if ind >= self.__len__() - 1:
            return self.__getitem__(0)
        return self.__getitem__(ind + 1)

    def skip_sample(self, ind):
        if self.shuffle:
            return self.random_sample()
        return self.sequential_sample(ind=ind)

    def __getitem__(self, ind):
        key = self.keys[ind]

        text_file = self.text_files[key]
        image_file = self.image_files[key]

        descriptions = text_file.read_text().split('\n')
        descriptions = list(filter(lambda t: len(t) > 0, descriptions))
        try:
            description = choice(descriptions)
        except IndexError as zero_captions_in_file_ex:
            print(f"An exception occurred trying to load file {text_file}.")
            print(f"Skipping index {ind}")
            return self.skip_sample(ind)

        tokenized_text = self.tokenizer.tokenize(
            description,
            self.text_len,
            truncate_text=self.truncate_captions
        ).squeeze(0)
        try:
            image_tensor = self.image_transform(PIL.Image.open(image_file))
        except (PIL.UnidentifiedImageError, OSError) as corrupt_image_exceptions:
            print(f"An exception occurred trying to load file {image_file}.")
            print(f"Skipping index {ind}")
            return self.skip_sample(ind)

        # Success
        return tokenized_text, image_tensor
    
ds = TextImageDataset(
        image_folder=IMAGE_FOLDER,
        text_folder=TEXT_FOLDER,
        text_len=TEXT_SEQ_LEN,
        image_size=IMAGE_SIZE,
        resize_ratio=RESIZE_RATIO,
        truncate_captions=TRUNCATE_CAPTIONS,
        tokenizer=tokenizer,
        shuffle=is_shuffle,
    )
assert len(ds) > 0, 'dataset is empty'

11789 image-text pairs found for training


In [32]:
if distr_backend.is_root_worker():
    if not ENABLE_WEBDATASET:
        print(f'{len(ds)} image-text pairs found for training')

11789 image-text pairs found for training


In [34]:
if not is_shuffle:
    data_sampler = torch.utils.data.distributed.DistributedSampler(
        ds,
        num_replicas=distr_backend.get_world_size(),
        rank=distr_backend.get_rank()
    )
else:
    data_sampler = None
data_sampler

In [35]:
dl = DataLoader(ds, batch_size=BATCH_SIZE, shuffle=is_shuffle, drop_last=True, sampler=data_sampler)

In [36]:
dalle = DALLE(vae=vae, **dalle_params)
if not using_deepspeed:
    if args.fp16:
        dalle = dalle.half()
    dalle = dalle.cuda()

if RESUME and not using_deepspeed:
    dalle.load_state_dict(weights)

In [37]:
opt = Adam(get_trainable_params(dalle), lr=LEARNING_RATE)
if RESUME and opt_state:
    opt.load_state_dict(opt_state)

if LR_DECAY:
    scheduler = ReduceLROnPlateau(
        opt,
        mode="min",
        factor=0.5,
        patience=10,
        cooldown=10,
        min_lr=1e-6,
        verbose=True,
    )
    if RESUME and scheduler_state:
        scheduler.load_state_dict(scheduler_state)
else:
    scheduler = None
    
scheduler

In [40]:
if distr_backend.is_root_worker():
    # experiment tracker

    model_config = dict(
        depth=DEPTH,
        heads=HEADS,
        dim_head=DIM_HEAD
    )

    run = wandb.init(
        project='dalle',
        entity='darayavaus',
        resume=RESUME,
        config=model_config,
    )
    run.name = f'training-transformer-{run.id}'

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: darayavaus (use `wandb login --relogin` to force relogin)


In [44]:
distr_backend.check_batch_size(BATCH_SIZE)
deepspeed_config = {
    'train_batch_size': BATCH_SIZE,
    'gradient_accumulation_steps': 1,
    'gradient_clipping': GRAD_CLIP_NORM,
    'fp16': {
        'enabled': FP16,
    },
    'amp': {
        'enabled': False,
        'opt_level': 'O1',
    },
    "flops_profiler": {
        "enabled": False,
        "profile_step": 200,
        "module_depth": -1,
        "top_modules": 1,
        "detailed": True,
        "output_file": None # TODO Can't get this to work.
    },
#     "zero_optimization": {
#         "stage": 3,
#         # Offload the model parameters If you have an nvme drive - you should use the nvme option.
#         # Otherwise, use 'cpu' and remove the `nvme_path` line
#         "offload_param": {
#             "device": "nvme",
#             "nvme_path": "/path/to/nvme/folder",
#         },
#         # Offload the optimizer of choice. If you have an nvme drive - you should use the nvme option.
#         # Otherwise, use 'cpu' and remove the `nvme_path` line
#         "offload_optimizer": {
#             "device": "nvme", # options are 'none', 'cpu', 'nvme'
#             "nvme_path": "/path/to/nvme/folder",
#         },
#     },
    "zero_optimization": {
        "stage": 3,
        "offload_optimizer": {
            "device": "cpu", 
        },
    },
}

if deepspeed_config.get('zero_optimization', {}).get('stage', 0) >= 2:
    print(f"Checkpoints made with DeepSpeed ZeRO Stages 2 and 3 will be stored in deepspeed checkpoint folder")
    print(f"As such, they will require DeepSpeed as a dependency in order to resume from or generate with.")
    print("See the deespeed conversion script for details on how to convert your ZeRO stage 2/3 checkpoint to a single file.")
    print("If using a single GPU, consider running with apex automatic mixed precision instead for a similar speedup to ZeRO.")
    time.sleep(2)

Checkpoints made with DeepSpeed ZeRO Stages 2 and 3 will be stored in deepspeed checkpoint folder
As such, they will require DeepSpeed as a dependency in order to resume from or generate with.
See the deespeed conversion script for details on how to convert your ZeRO stage 2/3 checkpoint to a single file.
If using a single GPU, consider running with apex automatic mixed precision instead for a similar speedup to ZeRO.


In [47]:
(distr_dalle, distr_opt, distr_dl, distr_scheduler) = distr_backend.distribute(
    args=args,
    model=dalle,
    optimizer=opt,
    model_parameters=get_trainable_params(dalle),
    training_data=(
        (None if ENABLE_WEBDATASET else ds)
        if using_deepspeed
        else dl
    ),
    # Do not pass the LR scheduler to DeepSpeed so we can manually
    # advance it.
    lr_scheduler=scheduler if LR_DECAY and not using_deepspeed else None,
    config_params=deepspeed_config,
)

[2021-10-11 04:43:28,866] [INFO] [logging.py:68:log_dist] [Rank 0] DeepSpeed info: version=0.5.4, git-hash=unknown, git-branch=unknown
[2021-10-11 04:43:28,917] [INFO] [engine.py:204:__init__] DeepSpeed Flops Profiler Enabled: False
[2021-10-11 04:43:28,918] [INFO] [engine.py:848:_configure_optimizer] Removing param_group that has no 'params' in the client Optimizer
[2021-10-11 04:43:28,918] [INFO] [engine.py:854:_configure_optimizer] Using client Optimizer as basic optimizer
[2021-10-11 04:43:28,919] [INFO] [engine.py:870:_configure_optimizer] DeepSpeed Basic Optimizer = Adam
[2021-10-11 04:43:28,919] [INFO] [utils.py:43:is_zero_supported_optimizer] Checking ZeRO support for optimizer=Adam type=<class 'torch.optim.adam.Adam'>
[2021-10-11 04:43:28,920] [INFO] [logging.py:68:log_dist] [Rank 0] Creating fp16 ZeRO stage 3 optimizer
Initializing ZeRO Stage 3
[2021-10-11 04:43:28,920] [INFO] [stage3.py:638:__init__] Reduce bucket size 500000000
[2021-10-11 04:43:28,920] [INFO] [stage3.py:63

[W ProcessGroupNCCL.cpp:1569] Rank 0 using best-guess GPU 0 to perform barrier as devices used by this process are currently unknown. This can potentially cause a hang if this rank to GPU mapping is incorrect.Specify device_ids in barrier() to force use of a particular device.


[2021-10-11 04:43:30,475] [INFO] [stage3.py:831:__init__] optimizer state initialized
[2021-10-11 04:43:30,492] [INFO] [logging.py:68:log_dist] [Rank 0] DeepSpeed Final Optimizer = Adam
[2021-10-11 04:43:30,492] [INFO] [engine.py:596:_configure_lr_scheduler] DeepSpeed using client LR scheduler
[2021-10-11 04:43:30,493] [INFO] [logging.py:68:log_dist] [Rank 0] DeepSpeed LR Scheduler = None
[2021-10-11 04:43:30,493] [INFO] [logging.py:68:log_dist] [Rank 0] step=0, skipped=0, lr=[0.0003], mom=[(0.9, 0.999)]
[2021-10-11 04:43:30,494] [INFO] [config.py:940:print] DeepSpeedEngine configuration:
[2021-10-11 04:43:30,495] [INFO] [config.py:944:print]   activation_checkpointing_config  {
    "partition_activations": false, 
    "contiguous_memory_optimization": false, 
    "cpu_checkpointing": false, 
    "number_checkpoints": null, 
    "synchronize_checkpoint_boundary": false, 
    "profile": false
}
[2021-10-11 04:43:30,496] [INFO] [config.py:944:print]   aio_config ................... {'blo

In [49]:
# Prefer scheduler in `deepspeed_config`.
if LR_DECAY and distr_scheduler is None:
    distr_scheduler = scheduler
avoid_model_calls = using_deepspeed and FP16

if RESUME and using_deepspeed:
    distr_dalle.load_checkpoint(str(cp_dir))

In [50]:
def save_model(path, epoch=0):
    save_obj = {
        'hparams': dalle_params,
        'vae_params': vae_params,
        'epoch': epoch,
    }
    if using_deepspeed:
        cp_dir = cp_path_to_dir(path, 'ds')

        if KEEP_N_CHECKPOINTS is not None and distr_backend.is_root_worker():
            checkpoints = sorted(glob(str(cp_dir / "global*")), key=os.path.getmtime, reverse=True)
            for checkpoint in checkpoints[KEEP_N_CHECKPOINTS:]:
                shutil.rmtree(checkpoint)

        distr_dalle.save_checkpoint(cp_dir, client_state=save_obj)

        if not distr_backend.is_root_worker():
            return

        # Save auxiliary values so we can reuse the standard routine
        # for loading.
        save_obj = {
            **save_obj,
            # Save a nonsense value that directs the user to
            # further help.
            'weights': (
                'To get a working standard checkpoint, '
                'look into consolidating DeepSpeed checkpoints.'
            ),
        }
        torch.save(save_obj, str(cp_dir / DEEPSPEED_CP_AUX_FILENAME))
        if deepspeed_config.get('zero_optimization', {}).get('stage', 0) >= 2: # see https://github.com/lucidrains/DALLE-pytorch/wiki/DeepSpeed-Checkpoints
            return

    if not distr_backend.is_root_worker():
        return

    save_obj = {
        **save_obj,
        'weights': dalle.state_dict(),
        'opt_state': opt.state_dict(),
    }
    save_obj['scheduler_state'] = (scheduler.state_dict() if scheduler else None)
    torch.save(save_obj, path)


In [60]:
# Saves a checkpoint before training begins to fail early when mis-configured.
# See https://github.com/lucidrains/DALLE-pytorch/wiki/DeepSpeed-Checkpoints
save_model(DALLE_OUTPUT_FILE_NAME, epoch=resume_epoch)

[2021-10-11 04:49:45,300] [INFO] [logging.py:68:log_dist] [Rank 0] Saving model checkpoint: ../models/dalle-birds-deepspeed-ds-cp/global_step0/zero_pp_rank_0_mp_rank_00_model_states.pt
[2021-10-11 04:49:46,209] [INFO] [engine.py:2492:_save_zero_checkpoint] zero checkpoint saved ../models/dalle-birds-deepspeed-ds-cp/global_step0/zero_pp_rank_0_mp_rank_00_optim_states.pt


In [66]:
for epoch in range(resume_epoch, EPOCHS):
    if data_sampler:
        data_sampler.set_epoch(epoch)
    for i, (text, images) in enumerate((dl if ENABLE_WEBDATASET else distr_dl)):
        if i % 10 == 0 and distr_backend.is_root_worker():
            t = time.time()
        if FP16:
            images = images.half()
        text, images = map(lambda t: t.cuda(), (text, images))

        loss = distr_dalle(text, images, return_loss=True)

        if using_deepspeed:
            distr_dalle.backward(loss)
            distr_dalle.step()
            # Gradients are automatically zeroed after the step
        else:
            loss.backward()
            clip_grad_norm_(distr_dalle.parameters(), GRAD_CLIP_NORM)
            distr_opt.step()
            distr_opt.zero_grad()

        # Collective loss, averaged
        avg_loss = distr_backend.average_all(loss)

        log = {}

        if i % 10 == 0 and distr_backend.is_root_worker():
            print(epoch, i, f'loss - {avg_loss.item()}')

            log = {
                **log,
                'epoch': epoch,
                'iter': i,
                'loss': avg_loss.item()
            }

        if i % SAVE_EVERY_N_STEPS == 0:
            save_model(DALLE_OUTPUT_FILE_NAME, epoch=epoch)

        if i % 100 == 0:
            if distr_backend.is_root_worker():
                sample_text = text[:1]
                token_list = sample_text.masked_select(sample_text != 0).tolist()
                decoded_text = tokenizer.decode(token_list)

                if not avoid_model_calls:
                    # CUDA index errors when we don't guard this
                    image = dalle.generate_images(text[:1], filter_thres=0.9)  # topk sampling at 0.9


                log = {
                    **log,
                }
                if not avoid_model_calls:
                    log['image'] = wandb.Image(image, caption=decoded_text)

        if i % 10 == 9 and distr_backend.is_root_worker():
            sample_per_sec = BATCH_SIZE * 10 / (time.time() - t)
            log["sample_per_sec"] = sample_per_sec
            print(epoch, i, f'sample_per_sec - {sample_per_sec}')

#         if i == 201 and args.flops_profiler:
#             raise StopIteration("Profiler has finished running. Stopping training early.")

        if distr_backend.is_root_worker():
            wandb.log(log)

    if LR_DECAY:
        distr_scheduler.step(avg_loss)

    save_model(DALLE_OUTPUT_FILE_NAME, epoch=epoch)
    
    if distr_backend.is_root_worker():
        # save trained model to wandb as an artifact every epoch's end

        model_artifact = wandb.Artifact('trained-dalle', type='model', metadata=dict(model_config))
        model_artifact.add_file(DALLE_OUTPUT_FILE_NAME)
        run.log_artifact(model_artifact)

0 0 loss - 9.397420883178711
[2021-10-11 04:54:09,546] [INFO] [logging.py:68:log_dist] [Rank 0] Saving model checkpoint: ../models/dalle-birds-deepspeed-ds-cp/global_step1/zero_pp_rank_0_mp_rank_00_model_states.pt
[2021-10-11 04:54:10,491] [INFO] [engine.py:2492:_save_zero_checkpoint] zero checkpoint saved ../models/dalle-birds-deepspeed-ds-cp/global_step1/zero_pp_rank_0_mp_rank_00_optim_states.pt


RuntimeError: CUDA out of memory. Tried to allocate 1.10 GiB (GPU 0; 10.76 GiB total capacity; 6.96 GiB already allocated; 187.44 MiB free; 9.44 GiB reserved in total by PyTorch)

In [ ]:
save_model(DALLE_OUTPUT_FILE_NAME, epoch=epoch)
if distr_backend.is_root_worker():
    wandb.save(DALLE_OUTPUT_FILE_NAME)
    model_artifact = wandb.Artifact('trained-dalle', type='model', metadata=dict(model_config))
    model_artifact.add_file(DALLE_OUTPUT_FILE_NAME)
    run.log_artifact(model_artifact)

    wandb.finish()

In [1]:
wandb.finish()

NameError: name 'wandb' is not defined